In [2]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn

import tensorflow as tf
tf.config.run_functions_eagerly(True)
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as layers

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [3]:
df = pd.read_csv('guitars.csv')

df

,brand,model,type,strings,frets,price,mensuration,body,colour,descr,img
0,Homage,HEG350BK,Stratocaster,6,22,14.110,25.500,массив твёрдого дерева,черный,C:/Users/user/ct/da/lab1-parser/descrs/1.html,C:/Users/user/ct/da/lab1-parser/imgs/1.jpg
1,ROCKET,ST-02 BB,Stratocaster,6,22,9.990,25.500,ольха,чёрный,C:/Users/user/ct/da/lab1-parser/descrs/2.html,C:/Users/user/ct/da/lab1-parser/imgs/2.jpg
2,ROCKET,EG-1 BK,Ibanez,6,22,11.990,25.500,ольха,чёрный,C:/Users/user/ct/da/lab1-parser/descrs/3.html,C:/Users/user/ct/da/lab1-parser/imgs/3.jpg
3,ROCKET,ST-32 SB,Stratocaster,6,22,7.990,25.500,липа,санбёрст,C:/Users/user/ct/da/lab1-parser/descrs/4.html,C:/Users/user/ct/da/lab1-parser/imgs/4.jpg
4,ROCKET,ST-01 SB,Stratocaster,6,22,9.490,25.500,ольха,чёрный,C:/Users/user/ct/da/lab1-parser/descrs/5.html,C:/Users/user/ct/da/lab1-parser/imgs/5.jpg
...,...,...,...,...,...,...,...,...,...,...,...
597,Fender,SQUIER CLASSIC VIBE STRAT 60`s 3-COLOR SUNBURST,Stratocaster,6,21,49.822,25.512,ольха,темное золото,C:/Users/user/ct/da/lab1-parser/descrs/598.html,C:/Users/user/ct/da/lab1-parser/imgs/598.jpg
598,Burny,RSG60`63 CR,SG,6,22,59.990,24.700,махагони,красный,C:/Users/user/ct/da/lab1-parser/descrs/599.html,C:/Users/user/ct/da/lab1-parser/imgs/599.jpg
599,Schecter,DAMIEN-6 SBK,Stratocaster,6,24,54.182,25.500,красное дерево,чёрный,C:/Users/user/ct/da/lab1-parser/descrs/600.html,C:/Users/user/ct/da/lab1-parser/imgs/600.jpg
600,Aria,PE-350PF AGBK,Les Paul,6,22,23.500,25.500,окоуме,agbk aged black,C:/Users/user/ct/da/lab1-parser/descrs/601.html,C:/Users/user/ct/da/lab1-parser/imgs/601.jpg


In [4]:
text1 = pd.DataFrame()
text1[['price', 'descr']] = df[['price', 'descr']]

text1

,price,descr
0,14.110,C:/Users/user/ct/da/lab1-parser/descrs/1.html
1,9.990,C:/Users/user/ct/da/lab1-parser/descrs/2.html
2,11.990,C:/Users/user/ct/da/lab1-parser/descrs/3.html
3,7.990,C:/Users/user/ct/da/lab1-parser/descrs/4.html
4,9.490,C:/Users/user/ct/da/lab1-parser/descrs/5.html
...,...,...
597,49.822,C:/Users/user/ct/da/lab1-parser/descrs/598.html
598,59.990,C:/Users/user/ct/da/lab1-parser/descrs/599.html
599,54.182,C:/Users/user/ct/da/lab1-parser/descrs/600.html
600,23.500,C:/Users/user/ct/da/lab1-parser/descrs/601.html


In [6]:
def opn(x):
    if x =="":
        return "_"
    with open(x, encoding='utf-8') as f: return f.read()
    
text = pd.DataFrame()
text['price'] = text1['price']
text['descr'] = text1['descr'].apply(lambda x: opn(x))

In [7]:
text

,price,descr
0,14.110,Homage HEG350 - электро-гитара под Telecaster....
1,9.990,Электрогитара ROCKET ST-02 (КОМПЛЕКТ - шнур Ja...
2,11.990,ROCKET EG-1 BK – мгновенно узнаваемой формы эл...
3,7.990,Детская электрогитара ROCKET ST-32/SB является...
4,9.490,Электрогитара Rocket ST-01 - это прекрасный ин...
...,...,...
597,49.822,Электрогитара
598,59.990,"Burny RSG60`63 CR электрогитара, форма корпус..."
599,54.182,"Серия гитар Damien имеет внешний вид и звук, п..."
600,23.500,"Aria PE-350PF сочетает в себе эстетику 60-х, д..."


In [8]:
target = pd.DataFrame()
target['price'] = text['price']
x = text.drop(columns=['price'])

train, test, train_price, test_price = train_test_split(x, target, test_size=0.2, random_state=8)
train, val, train_price, val_price = train_test_split(train, train_price, test_size=0.2/(1-0.2), random_state=8)

text.to_csv("text.csv")

In [9]:
vectorizer = tf.keras.layers.TextVectorization(max_tokens=1000, output_mode='int', output_sequence_length=100)
vectorizer.adapt(np.array(text.descr))

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


In [10]:
train_vectorized = vectorizer(train)

test_vectorized = vectorizer(test)

In [11]:
model = Sequential()
model.add(vectorizer)
model.add(layers.Embedding(10000, 100)) 
model.add(layers.LSTM(100)) 
model.add(layers.Dense(1))

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mse', 'mean_absolute_percentage_error'])

In [12]:
model.fit(train, train_price, validation_data=(val, val_price), epochs=35)

Epoch 1/35


C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


12/12 [==============================] - ETA: 0s - loss: 12439.6084 - mse: 12439.6084 - mean_absolute_percentage_error: 97.5948

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


12/12 [==============================] - 9s 765ms/step - loss: 12439.6084 - mse: 12439.6084 - mean_absolute_percentage_error: 97.5948 - val_loss: 13137.8721 - val_mse: 13137.8721 - val_mean_absolute_percentage_error: 88.2497
Epoch 2/35
12/12 [==============================] - 10s 812ms/step - loss: 11635.9199 - mse: 11635.9199 - mean_absolute_percentage_error: 82.5794 - val_loss: 12524.8799 - val_mse: 12524.8799 - val_mean_absolute_percentage_error: 76.6961
Epoch 3/35
12/12 [==============================] - 11s 905ms/step - loss: 11160.1885 - mse: 11160.1885 - mean_absolute_percentage_error: 73.4135 - val_loss: 12136.4941 - val_mse: 12136.4941 - val_mean_absolute_percentage_error: 69.6024
Epoch 4/35
12/12 [==============================] - 9s 806ms/step - loss: 10840.6719 - mse: 10840.6719 - mean_absolute_percentage_error: 67.7287 - val_loss: 11878.7354 - val_mse: 11878.7354 - val_mean_absolute_percentage_error: 65.4259
Epoch 5/35
12/12 [==============================] - 9s 788ms/step

In [13]:
y_test = model.predict(test)

C:\Users\user\AppData\Roaming\Python\Python311\site-packages\tensorflow\python\data\ops\structured_function.py:265: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


4/4 [==============================] - 1s 248ms/step


In [14]:
print(mean_squared_error(y_test, test_price))

9025.019595829723
